In [1]:
#pip install import-ipynb

In [2]:
from ipynb.fs.full.Functions_sec import *
import import_ipynb
import fuzzymatcher as fm 
import pandas as pd
import os

In [3]:
directory = os.getcwd()
path = directory + '/Data/'

# Reading files

In [4]:
header_list = ["cusip",'cik','titleOfClass','nameOfIssuer','putCall','investmentDiscretion','value','shrsOrPrnAmt.sshPrnamt',
               'shrsOrPrnAmt.sshPrnamtType','votingAuthority.Sole','votingAuthority.Shared','votingAuthority.None','id','Form']
cik_header=["nameOfIssuer","CIK"]

In [5]:
Filings=pd.read_csv(path + 'Filing_final.csv')
Hold_final=pd.read_csv(path + 'Hold_final.csv',names=header_list,low_memory=False, header=0)
cik_final=pd.read_csv(path + 'cik_final.csv',names=cik_header,header=0,sep=";")

## Matching company names of cik_lookup and Holdings

In [6]:
Filings["companyName_unchanged"]=Filings["companyName"]
Hold_final["nameOfIssuer_unchanged"]=Hold_final["nameOfIssuer"]
cik_final["nameOfIssuer_unchanged"]=cik_final["nameOfIssuer"]

In [7]:
Filings["companyName"]=transformation_v1(Filings,"companyName")    
Hold_final["nameOfIssuer"]=transformation_v1(Hold_final,"nameOfIssuer")    
cik_final["nameOfIssuer"]=transformation_v1(cik_final,"nameOfIssuer")    
Hold_final.rename(columns = {'cik':'cik_Filing'}, inplace = True)

In [8]:
Minus=cik_final[cik_final.duplicated('nameOfIssuer', keep="first")]
### Keeping first column only in case of duplicates on the column nameOfIssuer
cik_final=cik_final.drop(Minus.index).reset_index(drop=True)

## Finding cik of Holdings using Filings information

In [9]:
Hold_f=Hold_final.merge(Filings, how='left',left_on="nameOfIssuer",right_on="companyName")
For_cik_merge=Hold_f[Hold_f["cik"].isna()].iloc[: , :14]

In [10]:
len(Hold_final)

1910204

In [11]:
len(Hold_f[~Hold_f["cik"].isna()]) #out of 1910204, 45924 cik numbers have been filled by Filings file

45924

In [12]:
merge_1=Hold_f[~Hold_f["cik"].isna()][["cusip","cik_Filing","titleOfClass","putCall","cik","investmentDiscretion","value","nameOfIssuer","nameOfIssuer_unchanged"]]
merge_1.rename(columns = {'cik':'CIK'}, inplace = True)

## Round 1: Using cik lookup table to find cik numbers

In [13]:
Holdings_with_cik=pd.merge(For_cik_merge,cik_final,how="left", on ="nameOfIssuer")

In [14]:
len(Holdings_with_cik[Holdings_with_cik["CIK"].isna()])#1085881 cik have been filled by cik table, 778399 dont have cik number

778399

In [15]:
len(Holdings_with_cik[~Holdings_with_cik["CIK"].isna()])#1085921 cik have been filled by cik table, 778399 dont have cik number

1085881

In [16]:
cik_v2=pd.DataFrame(Holdings_with_cik[Holdings_with_cik["CIK"].isna()].iloc[:,:14])

In [17]:
merge_2=Holdings_with_cik[~Holdings_with_cik["CIK"].isna()][["cusip","cik_Filing","titleOfClass","putCall","CIK","investmentDiscretion","value","nameOfIssuer","nameOfIssuer_unchanged"]]

## Round 2 transformation: using regex

In [18]:
cik_final["nameOfIssuer"]=transformation_v2(cik_final,"nameOfIssuer")
cik_v2["nameOfIssuer"]=transformation_v2(cik_v2,"nameOfIssuer")

In [19]:
cik_v2=pd.merge(cik_v2,cik_final,how="left", on ="nameOfIssuer")

In [20]:
len(cik_v2[~cik_v2["CIK"].isna()]) #116768 were filled and 666179 are still empty

116768

In [21]:
len(cik_v2[cik_v2["CIK"].isna()])

666179

In [22]:
merge_3=cik_v2[~cik_v2["CIK"].isna()][["cusip","cik_Filing","titleOfClass","putCall","CIK","investmentDiscretion","value","nameOfIssuer","nameOfIssuer_unchanged"]]

##  Round 3 transformation: using regex

In [23]:
cik_v3=pd.DataFrame(cik_v2[cik_v2["CIK"].isna()])
cik_v3=cik_v3.drop(["CIK","nameOfIssuer_unchanged"],axis=1)

In [24]:
cik_v3["nameOfIssuer"]=transformation_v3(cik_v3,"nameOfIssuer")
cik_final["nameOfIssuer"]=transformation_v3(cik_final,"nameOfIssuer")

cik_v3["nameOfIssuer_1"]=cik_v3["nameOfIssuer"].replace(' ', '',regex=True)
cik_final["nameOfIssuer_1"]=cik_final["nameOfIssuer"].replace(' ', '',regex=True)

In [25]:
cik_v3=pd.merge(cik_v3,cik_final,how="left", on ="nameOfIssuer_1")

In [26]:
len(cik_v3[~cik_v3["CIK"].isna()])

101312

In [27]:
len(cik_v3[cik_v3["CIK"].isna()]) # 5,73,432 relationships dont have cik 
# some rows have zero cusip and 0 as value,
# 82 rows with zero cusip and value>0
# 573384 -82= 573302 rows dont have cik number
# 68500 companies dont have cik

573435

## Filtering records with CUSIP ="00000000"

In [28]:
cik_v3=cik_v3[~((cik_v3["cusip"]=="000000000") & (cik_v3["value"]==0)) ].reset_index(drop=True)

In [29]:
len(cik_v3[cik_v3["CIK"].isna()])

573387

In [30]:
merge_4=cik_v3[~cik_v3["CIK"].isna()][["cusip","cik_Filing","titleOfClass","putCall","CIK","investmentDiscretion","value","nameOfIssuer_x","nameOfIssuer_unchanged"]]
merge_4.rename(columns = {'nameOfIssuer_x':'nameOfIssuer'}, inplace = True)

In [31]:
Holdings_df=pd.concat((merge_1,merge_2,merge_3,merge_4),axis=0, ignore_index=True)

## Saving files

In [32]:
cik_v3.to_csv(path +'cik_v3.csv', index=False)
Holdings_df.to_csv(path+'Holdings_df.csv', index=False)
Filings.to_csv(path+'Filings_df.csv',index=False)